# Chat Intents

## Applying labels

In [122]:
import numpy as np
import pandas as pd

pd.set_option("display.max_rows", 600)
pd.set_option("display.max_columns", 500)
pd.set_option("max_colwidth", 400)

import collections
from sklearn.feature_extraction.text import CountVectorizer

import spacy

In [3]:
nlp = spacy.load("en_core_web_sm")

In [94]:
data_sample = pd.read_csv('../data/processed/data_sample.csv')
data_sample.sample(10)

,Unnamed: 0,text,category,cleaned_text
611,2200,I recently made a transfer but I need to cancel it as soon as possible. Please let me know when this happens.,cancel_transfer,recently made transfer need cancel soon possible please let know happens
234,8917,"I think someone might have a copy of my card because a 500 dollar withdrawal was made, and it wasn't me. Can you please help?",cash_withdrawal_not_recognised,think someone might copy card dollar withdrawal made wasnt please help
139,8660,I am still waiting for a transfer I did,balance_not_updated_after_bank_transfer,still waiting transfer
847,1130,I will be traveling. Can I add money automatically?,automatic_top_up,traveling add money automatically
647,884,Is it okay to hold money in multiple currencies?,fiat_currency_support,okay hold money multiple currency
87,8037,Can I top up online?,top_up_by_cash_or_cheque,top online
282,6179,"your company is awful, i would like to delete my account please",terminate_account,company awful would like delete account please
198,6191,How do I terminate my account.,terminate_account,terminate account
788,3535,I did not do this payment,card_payment_not_recognised,payment
229,1813,Are my contactless settings correct? I tried to use it today but it wasn't working.,contactless_not_working,contactless setting correct tried use today wasnt working


In [165]:
#example_category = data_sample[data_sample['category']=='cash_withdrawal_charge'].reset_index(drop=True)
example_category = data_sample[data_sample['category']=='cancel_transfer'].reset_index(drop=True)
example_category 

,Unnamed: 0,text,category,cleaned_text
0,2190,"I paid my rent into the wrong account, could you please pay it into the right account, it needs to be done by tomorrow.",cancel_transfer,paid rent wrong account could please pay right account need done tomorrow
1,2179,"Hey i need help urgently. I just realised I sent my payment to the wrong account number, there is a typo! Please cancel it immediately, I can't do it anymore from the app",cancel_transfer,hey need help urgently realised sent payment wrong account number typo please cancel immediately cant anymore app
2,2204,Please help me! A payment I made had an error on the account number and I can't cancel on my side. Can you cancel it?,cancel_transfer,please help payment made error account number cant cancel side cancel
3,2144,I just realized I made the wrong transaction yesterday. Would you please make it correct?I need it by tomorrow.,cancel_transfer,realized made wrong transaction yesterday would please make correcti need tomorrow
4,2129,Help! I need to cancel a transaction.,cancel_transfer,help need cancel transaction
5,2168,I am trying to revert a transcation I did this morning,cancel_transfer,trying revert transcation morning
6,2131,If I make a mistake can I cancel the transaction?,cancel_transfer,make mistake cancel transaction
7,2180,is it possible to cancel transactions?,cancel_transfer,possible cancel transaction
8,2078,Am I able to cancel a transfer I just made,cancel_transfer,able cancel transfer made
9,2108,I need to cancel a transfer I made.,cancel_transfer,need cancel transfer made


In [166]:
example_doc = nlp(example_category.loc[0,'text'])
for token in example_doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_ , token.is_stop)

I I PRON PRP nsubj True
paid pay VERB VBD ccomp False
my my PRON PRP$ poss True
rent rent NOUN NN dobj False
into into ADP IN prep True
the the DET DT det True
wrong wrong ADJ JJ amod False
account account NOUN NN pobj False
, , PUNCT , punct False
could could AUX MD aux True
you you PRON PRP nsubj True
please please INTJ UH intj True
pay pay VERB VB ccomp False
it it PRON PRP dobj True
into into ADP IN prep True
the the DET DT det True
right right ADJ JJ amod False
account account NOUN NN pobj False
, , PUNCT , punct False
it it PRON PRP nsubj True
needs need VERB VBZ ROOT False
to to PART TO aux True
be be AUX VB auxpass True
done do VERB VBN xcomp True
by by ADP IN prep True
tomorrow tomorrow NOUN NN pobj False
. . PUNCT . punct False


In [167]:
len(example_category)

23

In [160]:
data_sample['category'].unique()

array(['card_delivery_estimate', 'cash_withdrawal_charge',
       'balance_not_updated_after_cheque_or_cash_deposit',
       'receiving_money', 'edit_personal_details', 'card_arrival',
       'card_acceptance', 'fiat_currency_support',
       'declined_cash_withdrawal', 'country_support',
       'topping_up_by_card', 'wrong_exchange_rate_for_cash_withdrawal',
       'Refund_not_showing_up', 'getting_virtual_card',
       'visa_or_mastercard', 'balance_not_updated_after_bank_transfer',
       'automatic_top_up', 'age_limit', 'transfer_fee_charged',
       'card_not_working', 'lost_or_stolen_phone',
       'verify_source_of_funds', 'card_about_to_expire', 'verify_top_up',
       'top_up_reverted', 'pending_top_up', 'top_up_failed',
       'card_payment_fee_charged', 'atm_support',
       'pending_cash_withdrawal', 'pending_card_payment',
       'cancel_transfer', 'top_up_by_cash_or_cheque',
       'getting_spare_card', 'request_refund',
       'direct_debit_payment_not_recognised', 'disp

In [161]:
def get_group(df):
    
    example_category = data_sample[data_sample['category']=='cancel_transfer'].reset_index(drop=True)
example_category 

,Unnamed: 0,text,category,cleaned_text
0,5714,Transaction refund missing from my statement.,Refund_not_showing_up,transaction refund missing statement
1,5765,I contacted the seller and he won't respond. Please just refund me my money.,Refund_not_showing_up,contacted seller wont respond please refund money
2,5799,I am missing my refund.,Refund_not_showing_up,missing refund
3,5679,Why is my refund not showing up on the statement?,Refund_not_showing_up,refund showing statement
4,5776,how does my statement get updated so i know when a refund will reflect on my balance,Refund_not_showing_up,statement get updated know refund reflect balance
5,5807,"I'm looking at my statement, and my refund is not on there.",Refund_not_showing_up,im looking statement refund
6,5674,"I don't see my refund money yet in my account. I asked the merchant a while ago for a refund, so I don't know what's taking so long. Can you look into it and see if the money is there, just not in my account yet?",Refund_not_showing_up,dont see refund money yet account asked merchant ago refund dont know whats taking long look see money account yet
7,5786,I cannot view the refunded amount in my account.,Refund_not_showing_up,cannot view refunded amount account
8,5691,I keep checking my statements but nothing shows up. I requested a refund a long time ago but seems it's not reaching my account for some reason. Would you please help me get the money back from the seller.,Refund_not_showing_up,keep checking statement nothing show requested refund long time ago seems reaching account reason would please help get money back seller
9,5681,why cant you just refund me instead of a merchant,Refund_not_showing_up,cant refund instead merchant


In [162]:
def most_common(lst, n_words):
    counter=collections.Counter(lst)
    return counter.most_common(n_words)

In [163]:
def extract_labels(category_docs, print_word_counts=False):
    """
    Argument:
    category_docs: list of documents, all from the same category or clustering
    """
    verbs = []
    dobjs = []
    nouns = []
    adjs = []

    for i in range(len(category_docs)):
        doc = nlp(category_docs.loc[i,'text'])
        for token in doc:
            if token.is_stop==False:
                if token.dep_ == 'ROOT':
                    verbs.append(token.lemma_)

                elif token.dep_=='dobj':
                    dobjs.append(token.lemma_)

                elif token.pos_=='NOUN':
                    nouns.append(token.lemma_)
                    
                elif token.pos_=='ADJ':
                    adjs.append(token.lemma_)
                    #print(i, token.text, token.pos_, token.tag_, token.dep_ )

    if print_word_counts:
        for word_lst in [verbs, dobjs, nouns, adjs]:
            counter=collections.Counter(word_lst)
            print(counter)
        
    verb = most_common(verbs, 1)[0][0]
    dobj = most_common(dobjs, 1)[0][0]
    noun1 = most_common(nouns, 1)[0][0]
    noun2 = most_common(nouns, 2)[1][0]
    
    words = [verb, dobj]
    
    for word in [noun1, noun2]:
        if word not in words:
            words.append(word)
    
    label = '_'.join(words)
    
    return label

In [168]:
extract_labels(example_category, True)

Counter({'need': 7, 'cancel': 7, 'help': 3, 'realise': 1, 'realize': 1, 'try': 1, 'want': 1, 'let': 1})
Counter({'transfer': 8, 'transaction': 6, 'payment': 4, 'rent': 1, 'help': 1, 'error': 1, 'transcation': 1, 'mistake': 1})
Counter({'account': 9, 'tomorrow': 3, 'number': 3, 'payment': 3, 'yesterday': 3, 'transaction': 2, 'morning': 2, 'rent': 2, 'typo': 1, 'app': 1, 'correct?i': 1})
Counter({'wrong': 6, 'possible': 3, 'right': 2, 'able': 2, 'large': 1, 'important': 1, 'incorrect': 1, 'recent': 1})


'need_transfer_account_tomorrow'

In [100]:
extract_labels(example_category)

Counter({'need': 2, 'cards': 1, 'tried': 1, 'work': 1})
Counter({'cards': 4, 'card': 2})
Counter({'card': 6})
Counter({'cards': 3, 'limit': 1, 'times': 1, 'person': 1, 'hotel': 1, 'rules': 1, 'limits': 1, 'day': 1, 'downsides': 1})


'need_cards_limit'

In [34]:
counter=collections.Counter(verbs)
print(counter)

Counter({'charged': 12, 'is': 4, 'charge': 2, 'took': 1, 'going': 1, 'seen': 1, 'charging': 1, 'went': 1, "'s": 1, 'was': 1, 'came': 1, 'have': 1, 'Is': 1, 'supposed': 1, 'had': 1, 'changed': 1, 'expect': 1, 'realize': 1, 'are': 1, 'cost': 1, 'Explain': 1})


In [38]:
counter=collections.Counter(dobjs)
print(counter)

Counter({'fee': 8, 'cash': 6, 'money': 3, 'ATM': 3, 'withdrawal': 1, 'charge': 1, 'fees': 1, 'groceries': 1, 'transaction': 1, 'look': 1, 'it': 1, 'me': 1, 'anything': 1})


In [51]:
counter.most_common(2)

[('fee', 8), ('cash', 6)]

In [118]:
#all_messages_per_topic = intents_df.groupby(['topic'], as_index=False).agg({'message': ' '.join})
all_messages_per_topic = data_sample.groupby(['category'], as_index=False).agg({'text': ' '.join})

In [138]:
def c_tf_idf(documents, m, ngram_range=(1,1)):
    count = CountVectorizer(ngram_range = ngram_range, stop_words = 'english').fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)
    
    return tf_idf, count

def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=10):
    words = count.get_feature_names()
    labels = list(docs_per_topic.category)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    top_n_words_only = {label: [words[j] for j in indices[i]][::-1] for i, label in enumerate(labels)}
    
    return top_n_words, top_n_words_only

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['category']).text.count()
                   .reset_index()
                   .rename({'text':'size'}, axis='columns')
                   .sort_values('size', ascending=False))
    return topic_sizes

In [139]:
tf_idf, count = c_tf_idf(all_messages_per_topic.text.values, m = len(data_sample))

In [149]:
top_n_words, top_n_words_only = extract_top_n_words_per_topic(tf_idf, count, all_messages_per_topic, n=4)

In [154]:
top_n_words['wrong_amount_of_cash_received']

[('withdraw', 0.31597398480451466),
 ('cash', 0.2646703656236372),
 ('100', 0.2481555468702133),
 ('20', 0.2481555468702133)]

In [155]:
#top_n_words[-1]

In [152]:
df_keywords = pd.DataFrame()
df_keywords['category'] = top_n_words_only.keys()
df_keywords['keywords'] = top_n_words_only.values()

In [153]:
extract_topic_sizes(data_sample).merge(df_keywords, how='left', on='category')

,category,size,keywords
0,Refund_not_showing_up,26,"(refund, statement, requested, ago)"
1,cash_withdrawal_charge,24,"(charged, cash, fee, withdrawal)"
2,cancel_transfer,23,"(cancel, transaction, need, wrong)"
3,pending_card_payment,23,"(payment, pending, long, does)"
4,transfer_not_received_by_recipient,22,"(friend, money, transaction, sent)"
5,direct_debit_payment_not_recognised,22,"(debit, direct, didn, app)"
6,request_refund,21,"(refund, item, bought, return)"
7,cash_withdrawal_not_recognised,21,"(withdrawal, account, help, 500)"
8,verify_top_up,18,"(verify, verification, code, need)"
9,verify_source_of_funds,18,"(funds, source, know, money)"


In [ ]:
intents_df[intents_df['message'].str.contains('card')]